In [7]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
from helper_functions import *

In [2]:
classification_types = ["world_news", "finance", "sports", "politics", "local_news"]
response_format = get_response_format(classification_types)


json_output_instance = {
    "response": {
            "reasoning": "This news article is of the type internal_news because of bla bla bla",
            "classification": "internal_news"
    }
}

system_prompt = f"""You are a journalist with over 25 years of experience with great expertise in reading and classifying news articles.
You will receive the content of a news article and your task is to classify it into a set of pre-defined news types.
Please read the entire article in full and then respond in the following format:
{format_json(json_output_instance)}
Add your reasoning to the "reasoning" entry.
Add the classification to the "classification" entry.
"""

In [3]:
text_input = """
This is an article about the US political system
"""

In [5]:
response = get_response(system_prompt, text_input, response_format)
print(format_json(response))

{
  "reasoning": "This news article discusses aspects of the US political system, aligning with governance, laws, and political issues, which fits under the category of politics.",
  "classification": "politics"
}
